This creates timing files for afni. It will make ones for:

* faces (onsets)
* famous (vs non-famous) faces (onsets for famous and non-famous)
* repetition of the same video (ensure to scale...separate by famous/non-famous?)
* repetition of the exemplar (ensure to scale)

Let's do the first two first.

In [90]:
import os
import pandas as pd
import numpy as np
import readline
import rpy2
import rpy2.robjects as robjects
r = robjects.r

import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

# Run

In [101]:
# Skip the first subject...for now
for si in range(1,6):
    subj = "sub%02i" % (si+1)
    print(subj)
    
    # Load the R data
    infile = "/data1/famface01/analysis/encoding/ShapeAnalysis/data/fam_roi_n_more_%s.rda" % subj
    r.load(infile)
    
    # Variables
    onsets     = np.array(r.dat.rx2('basics').rx2('timing').rx2('onset'))
    questions  = np.array(r['as.character'](r.dat.rx2('basics').rx2('timing').rx2('question')))
    vids       = np.array(r['as.character'](r.dat.rx2('basics').rx2('timing').rx2('video')))
    vidlevs    = np.array(r.dat.rx2('basics').rx2('timing').rx2('video').levels)
    fams       = np.array(r.dat.rx2('basics').rx2('timing').rx2('fam'))
    runs       = np.array(r.dat.rx2('basics').rx2('timing').rx2('run'))
    uruns      = np.unique(runs)

    # Get repeated vids
    vid_reps = np.zeros(len(vids))
    for vname in vidlevs:
        vid_reps[vids == vname] = [1,2]
    
    # Person repeats
    import re
    persons = np.array([ re.sub("_num[0-9]{2}", "", vname) for vname in vids ])
    upersons= np.unique(persons)
    person_reps = np.zeros(len(persons))
    for pname in upersons:
        person_reps[persons == pname] = range(1,13)

    
    ###
    # FACE ACTIVITY
    ###

    nruns = uruns.shape[0]
    afni_facemat = []

    for ri in range(nruns):
        run_inds = runs == uruns[ri]
        n = np.sum(run_inds)

        ovec = onsets[run_inds].astype('float32').round(4)
        row = [ '%.2f' % ovec[i] for i in range(n) ]
        row = " ".join(row)

        afni_facemat.append(row)

    afni_facemat = np.array(afni_facemat)
    
    
    ###
    # QUESTIONS
    ###

    q_regressor = (questions != 'none') * 1

    afni_qmat = []
    nruns = uruns.shape[0]
    for ri in range(nruns):
        run_inds = runs == uruns[ri]
        n = np.sum(run_inds)

        qvec = q_regressor[run_inds]
        ovec = onsets[run_inds].astype('float32').round(4)
        row  = np.array([ '%.2f' % ovec[i] for i,touse in enumerate(qvec) if touse == 1 ])
        if len(row) == 0:
            row = '*'
        else:
            row = " ".join(row)

        afni_qmat.append(row)
    
    
    ###
    # FAMOUS/NON-FAMOUS FACE ACTIVITY
    ###

    nruns = uruns.shape[0]

    # Famous
    afni_fammat = []
    for ri in range(nruns):
        run_inds = (runs == uruns[ri]) & (fams == "yes")
        n = np.sum(run_inds)
        ovec = onsets[run_inds].astype('float32').round(4)
        row = [ '%.2f' % ovec[i] for i in range(n) ]
        row = " ".join(row)
        afni_fammat.append(row)
    afni_fammat = np.array(afni_fammat)

    # NonFamous
    afni_nonfammat = []
    for ri in range(nruns):
        run_inds = (runs == uruns[ri]) & (fams == "no")
        n = np.sum(run_inds)
        ovec = onsets[run_inds].astype('float32').round(4)
        row = [ '%.2f' % ovec[i] for i in range(n) ]
        row = " ".join(row)
        afni_nonfammat.append(row)
    afni_nonfammat = np.array(afni_nonfammat)
    
    
    ###
    # MOTION
    ###

    funcdir = "/data1/famface01/analysis/preprocessed/%s/func" % subj
    df_paths = pd.read_table("%s/df_paths.txt" % funcdir, sep=" ")

    inds = df_paths.inindex[df_paths.name == 'fam_vids']
    motion_fpaths = [ "%s/mc/func_run%02i_dfile.1D" % (funcdir, ind) for ind in inds ]

    from sklearn.preprocessing import scale
    motion_mats = []
    for fpath in motion_fpaths:
        x = np.loadtxt(fpath)
        x = scale(x, with_std=False, with_mean=True)
        motion_mats.append(x)
    motion_mat = np.vstack(motion_mats)
    
    
    ###
    # SAVE
    ###

    base = "/data1/famface01/command/misc/face_representations"
    outbase = "%s/300_task_activity/200_famous_face_basics_fam/timings" % base
    outdir = "%s/%s" % (outbase, subj)
    print outdir
    if not os.path.exists(outdir):
        os.mkdir(outdir)

    # Just faces
    ofname = '%s/stim_faces.txt' % outdir
    np.savetxt(ofname, afni_facemat, fmt='%s')

    # Famous
    ofname = '%s/stim_famous.txt' % outdir
    np.savetxt(ofname, afni_fammat, fmt='%s')
    ofname = '%s/stim_nonfamous.txt' % outdir
    np.savetxt(ofname, afni_nonfammat, fmt='%s')

    # Questions
    ofname = '%s/stim_questions.txt' % outdir
    np.savetxt(ofname, afni_qmat, fmt='%s')

    # MOTION
    ofname = '%s/motion.1D' % outdir
    np.savetxt(ofname, motion_mat, fmt='%f')

sub02
/data1/famface01/command/misc/face_representations/300_task_activity/200_famous_face_basics_fam/timings/sub02
sub03
/data1/famface01/command/misc/face_representations/300_task_activity/200_famous_face_basics_fam/timings/sub03
sub04
/data1/famface01/command/misc/face_representations/300_task_activity/200_famous_face_basics_fam/timings/sub04
sub05
/data1/famface01/command/misc/face_representations/300_task_activity/200_famous_face_basics_fam/timings/sub05
sub06
/data1/famface01/command/misc/face_representations/300_task_activity/200_famous_face_basics_fam/timings/sub06


# Scratch

In [59]:
subj = "sub03"

# Load the R data
infile = "/data1/famface01/analysis/encoding/ShapeAnalysis/data/fam_roi_n_more_%s.rda" % subj
print(infile)
r.load(infile)

print(r.dat.rx2('basics').rx2('timing').names)

/data1/famface01/analysis/encoding/ShapeAnalysis/data/fam_roi_n_more_sub03.rda
[1] "run"         "onset"       "local.onset" "duration"    "question"   
[6] "video"       "fam"        



In [60]:
onsets     = np.array(r.dat.rx2('basics').rx2('timing').rx2('onset'))
questions  = np.array(r['as.character'](r.dat.rx2('basics').rx2('timing').rx2('question')))
vids       = np.array(r['as.character'](r.dat.rx2('basics').rx2('timing').rx2('video')))
vidlevs    = np.array(r.dat.rx2('basics').rx2('timing').rx2('video').levels)
fams       = np.array(r.dat.rx2('basics').rx2('timing').rx2('fam'))
runs       = np.array(r.dat.rx2('basics').rx2('timing').rx2('run'))
uruns      = np.unique(runs)

print(r.table(questions))
print(r.table(runs))


        age  attractive      gender intelligent        none trustworthy 
          8           8          16           8         720           8 


 1  2  3  4  5  6  7  8 
96 96 96 96 96 96 96 96 



In [61]:
# Now we want to get if the same video repeated or not
vid_reps = np.zeros(len(vids))
for vname in vidlevs:
    vid_reps[vids == vname] = [1,2]
vid_reps[:200]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1

In [70]:
# And now we want when the same person's video repeats
import re

persons = np.array([ re.sub("_num[0-9]{2}", "", vname) for vname in vids ])
upersons= np.unique(persons)

person_reps = np.zeros(len(persons))
for pname in upersons:
    person_reps[persons == pname] = range(1,13)
person_reps[:200]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        2.,  1.,  1.,  2.,  1.,  1.,  2.,  2.,  2.,  1.,  1.,  1.,  2.,
        1.,  1.,  2.,  2.,  2.,  3.,  1.,  2.,  2.,  2.,  1.,  2.,  2.,
        1.,  1.,  2.,  2.,  1.,  1.,  1.,  2.,  3.,  2.,  3.,  2.,  1.,
        3.,  2.,  2.,  3.,  2.,  3.,  3.,  1.,  3.,  3.,  3.,  3.,  2.,
        2.,  2.,  3.,  3.,  2.,  2.,  2.,  3.,  2.,  3.,  1.,  3.,  2.,
        3.,  3.,  2.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  2.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  4.,  4.,  4.,  4.,  4.,  5.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  5.,  4.,  4.,  4.,  5.,  4.,  4.,  5.,
        4.,  5.,  5.,  4.,  5.,  4.,  4.,  5.,  4.,  5.,  5.,  4.,  6.,
        4.,  4.,  5.,  5.,  4.,  4.,  5.,  4.,  5.,  5.,  4.,  4.,  5.,
        5.,  5.,  5.,  4.,  6.,  6.,  5.,  5.,  6.,  6.,  5.,  5.,  6.,
        6.,  6.,  6.,  5.,  6.,  5.,  6.,  6.,  6.,  6.,  6.,  5.,  5.,
        4.,  6.,  5.,  6.,  5.,  6.,  5.,  6.,  6.,  6.,  5.,  6

In [75]:
###
# FACE ACTIVITY
###

nruns = uruns.shape[0]
afni_facemat = []

for ri in range(nruns):
    run_inds = runs == uruns[ri]
    n = np.sum(run_inds)
    
    ovec = onsets[run_inds].astype('float32').round(4)
    row = [ '%.2f' % ovec[i] for i in range(n) ]
    row = " ".join(row)

    afni_facemat.append(row)

afni_facemat = np.array(afni_facemat)
afni_facemat[:2]

array([ '0.00 2.00 4.00 8.00 10.00 12.00 14.00 16.00 18.00 22.00 26.00 28.00 30.00 34.00 44.00 48.00 54.00 56.00 58.00 60.00 64.00 66.00 70.00 72.00 74.00 78.00 80.00 82.00 84.00 86.00 88.00 90.00 94.00 96.00 98.00 104.00 106.00 108.00 112.00 116.00 118.00 122.00 124.00 126.00 128.00 130.00 134.00 136.00 140.00 142.00 144.00 146.00 150.00 152.00 154.00 156.00 160.00 162.00 168.00 170.00 176.00 178.00 180.00 184.00 186.00 188.00 190.00 192.00 196.00 200.00 204.00 206.00 208.00 210.00 218.00 222.00 224.00 228.00 232.00 234.00 236.00 240.00 242.00 246.00 248.00 250.00 252.00 254.00 258.00 264.00 266.00 268.00 272.00 274.00 276.00 278.00',
       '286.00 288.00 290.00 292.00 294.00 296.00 300.00 302.00 304.00 306.00 308.00 310.00 316.00 322.00 324.00 326.00 328.00 330.00 334.00 336.00 338.00 340.00 346.00 348.00 352.00 354.00 358.00 360.00 362.00 366.00 368.00 370.00 374.00 378.00 380.00 384.00 386.00 388.00 394.00 398.00 400.00 402.00 406.00 408.00 410.00 414.00 416.00 418.00 424.00 426.0

In [79]:
###
# QUESTIONS
###

q_regressor = (questions != 'none') * 1
print(r.table(q_regressor))

## note: this will need to be saved separately

afni_qmat = []
nruns = uruns.shape[0]
for ri in range(nruns):
    run_inds = runs == uruns[ri]
    n = np.sum(run_inds)

    qvec = q_regressor[run_inds]
    ovec = onsets[run_inds].astype('float32').round(4)
    row  = np.array([ '%.2f' % ovec[i] for i,touse in enumerate(qvec) if touse == 1 ])
    if len(row) == 0:
        row = '*'
    else:
        row = " ".join(row)
    
    afni_qmat.append(row)

afni_qmat[:2]


  0   1 
720  48 



['34.00 48.00 98.00 162.00 210.00 258.00',
 '310.00 340.00 388.00 448.00 496.00 542.00']

In [82]:
###
# FAMOUS/NON-FAMOUS FACE ACTIVITY
###

nruns = uruns.shape[0]

# Famous
afni_fammat = []
for ri in range(nruns):
    run_inds = (runs == uruns[ri]) & (fams == "yes")
    n = np.sum(run_inds)
    ovec = onsets[run_inds].astype('float32').round(4)
    row = [ '%.2f' % ovec[i] for i in range(n) ]
    row = " ".join(row)
    afni_fammat.append(row)
afni_fammat = np.array(afni_fammat)

# NonFamous
afni_nonfammat = []
for ri in range(nruns):
    run_inds = (runs == uruns[ri]) & (fams == "no")
    n = np.sum(run_inds)
    ovec = onsets[run_inds].astype('float32').round(4)
    row = [ '%.2f' % ovec[i] for i in range(n) ]
    row = " ".join(row)
    afni_nonfammat.append(row)
afni_nonfammat = np.array(afni_nonfammat)

print afni_fammat[:2]
print afni_nonfammat[:2]

[ '0.00 2.00 4.00 8.00 10.00 12.00 14.00 16.00 18.00 26.00 28.00 30.00 34.00 44.00 48.00 54.00 56.00 58.00 60.00 66.00 72.00 74.00 78.00 80.00 82.00 84.00 86.00 88.00 90.00 94.00 96.00 98.00 104.00 106.00 108.00 112.00 116.00 118.00 124.00 126.00 128.00 130.00 134.00 136.00 140.00 142.00 144.00 146.00 150.00 152.00 154.00 156.00 160.00 162.00 168.00 176.00 178.00 184.00 186.00 188.00 190.00 192.00 196.00 200.00 204.00 206.00 208.00 210.00 218.00 224.00 228.00 234.00 240.00 242.00 246.00 248.00 250.00 254.00 264.00 266.00 268.00 272.00 276.00 278.00'
 '286.00 288.00 290.00 292.00 294.00 296.00 300.00 302.00 304.00 306.00 308.00 310.00 316.00 322.00 324.00 328.00 330.00 334.00 336.00 338.00 340.00 346.00 352.00 354.00 362.00 366.00 368.00 370.00 374.00 378.00 380.00 384.00 386.00 388.00 398.00 400.00 402.00 406.00 410.00 414.00 416.00 418.00 424.00 428.00 430.00 432.00 436.00 438.00 440.00 442.00 448.00 454.00 456.00 462.00 464.00 468.00 470.00 472.00 476.00 478.00 486.00 490.00 496.00 5

In [87]:
###
# MOTION
###

funcdir = "/data1/famface01/analysis/preprocessed/%s/func" % subj
df_paths = pd.read_table("%s/df_paths.txt" % funcdir, sep=" ")

inds = df_paths.inindex[df_paths.name == 'fam_vids']
motion_fpaths = [ "%s/mc/func_run%02i_dfile.1D" % (funcdir, ind) for ind in inds ]

from sklearn.preprocessing import scale
motion_mats = []
for fpath in motion_fpaths:
    x = np.loadtxt(fpath)
    x = scale(x, with_std=False, with_mean=True)
    motion_mats.append(x)
motion_mat = np.vstack(motion_mats)

In [88]:
motion_mat

array([[-0.19022832, -0.14823951, -0.12488741,  0.06031958, -0.13491189,
        -0.02307552],
       [-0.17122832, -0.08213951, -0.14778741,  0.04431958, -0.12501189,
         0.06402448],
       [-0.15632832, -0.06123951, -0.10048741, -0.00618042, -0.11831189,
         0.05532448],
       ..., 
       [ 0.00352902,  0.0941958 ,  0.03237867, -0.11997308, -0.01660839,
         0.03874336],
       [ 0.00622902,  0.1072958 ,  0.02277867, -0.14177308, -0.00790839,
         0.03494336],
       [ 0.00742902,  0.0340958 ,  0.01687867, -0.02637308, -0.00930839,
        -0.05065664]])

In [93]:
###
# SAVE
###

base = "/data1/famface01/command/misc/face_representations"
outbase = "%s/300_task_activity/200_famous_face_basics_fam/timings" % base
outdir = "%s/%s" % (outbase, subj)
print outdir
if not os.path.exists(outdir):
    os.mkdir(outdir)

# Just faces
ofname = '%s/stim_faces.txt' % outdir
np.savetxt(ofname, afni_facemat, fmt='%s')

# Famous
ofname = '%s/stim_famous.txt' % outdir
np.savetxt(ofname, afni_fammat, fmt='%s')
ofname = '%s/stim_nonfamous.txt' % outdir
np.savetxt(ofname, afni_nonfammat, fmt='%s')

# Questions
ofname = '%s/stim_questions.txt' % outdir
np.savetxt(ofname, afni_qmat, fmt='%s')

# MOTION
ofname = '%s/motion.1D' % outdir
np.savetxt(ofname, motion_mat, fmt='%f')

/data1/famface01/command/misc/face_representations/300_task_activity/200_famous_face_basics_fam/timings/sub03
